In [ ]:
@staticmethod
    def model_baseline_const3(model, inputs):
        
        """
        Adds constraints to the given mathematical optimization `model` based on the provided `inputs`.

        This function adds constraints to the optimization model to ensure
            - Inbound volume:
                Is about the volume indicated by "CycleVolIn2"
                The flow rate satisfies the bounds indicated by "Bounds"
            - Outbound volume
                Is about the volume indicated by "CycleVolOut2"
                The flow rate satisfies the bounds indicated by "Bounds"
            - Zero flows:
                Zero outflows for the state space that does not belong to CycleVolOut2

        Parameters:
            model (OptimizationModel): The mathematical optimization model to which constraints are added.
            inputs (dict): A dictionary containing input data required to create constraints.
                The dictionary should have the following keys:
                    - 'index': A dictionary containing information about 'i' and 'o' indexes.
                        It should have the following keys:
                            - 'i': A list of tuples representing the 'i' index with format (tank, line, prod, j, ...)
                            - 'o': A list of tuples representing the 'o' index with format (tank, line, prod, j, ...)
                    - 'i', 'p': The optimization variables representing inflow in the model.
                    - 'o', 'q': The optimization variables representing outflow in the model.
                    - 'CycleVolIn2': A dictionary containing cycle volume for inflow operations.
                    - 'CycleVolOut2': A dictionary containing cycle volume for outflow operations.
                    - 'Bounds': A dictionary containing line flow rate bounds.

        Returns:
            OptimizationModel: The `model` with added constraints.

        Example:
            # Assume 'model' and 'inputs' are defined.
            model = function_model_const3(model, inputs)

        Note:
            - The 'i', 'o', 'p', and 'q' optimization variables must be previously defined in the model before calling this function.
            - The function will create constraints for cycle volume constraints of inflow and outflow operations specified in 'CycleVolIn2' and 'CycleVolOut2'.
            - The function will create constraints based on bounds specified in 'Bounds' for both inflow and outflow operations.
            - If a line and product combination specified in 'CycleVolOut2' has no value (None), the corresponding outflow and consumption will be set to 0.
        """

        i_index, o_index, i, o, p, q, CycleVolIn2, CycleVolOut2, Bounds = (
            inputs['index']['i'],
            inputs['index']['o'],
            inputs['i'],
            inputs['o'],
            inputs['p'],
            inputs['q'],
            inputs['CycleVolIn2'],
            inputs['CycleVolOut2'],
            inputs['Bounds']
        )

        #-------------------------------------------------------------------------------------------------------------              
        # Inflow Constraints
        #
        for line in CycleVolIn2:
            for prod in CycleVolIn2[line]:
                model.addConstr(i.sum("*", line, prod, "*") >= CycleVolIn2[line][prod])
                model.addConstr(i.sum("*", line, prod, "*") <= CycleVolIn2[line][prod] + 30)
        
        lst = list(set([t[0:4] for t in i_index]))            
        for tup in lst:
            tank = tup[0]; line = tup[1]; prod = tup[2]; j = tup[3]
            model.addConstr(i[tank, line, prod, j] - p[tank, line, prod, j] * Bounds[line]["u"]  <= 0)
            model.addConstr(i[tank, line, prod, j] - p[tank, line, prod, j] * Bounds[line]["l"]  >= 0) 
            
        #-------------------------------------------------------------------------------------------------------------                        
        # Outflow constraints
        #
        for line in CycleVolOut2:
            for prod in CycleVolOut2[line]:
                model.addConstr(o.sum("*", line, prod, "*") >= CycleVolOut2[line][prod] )            
                model.addConstr(o.sum("*", line, prod, "*") <= CycleVolOut2[line][prod] + 10)            
        
        lst = list(set([t[0:4] for t in o_index]))            
        for tup in lst:
            tank = tup[0]; line = tup[1]; prod = tup[2]; j = tup[3]
            model.addConstr(o[tank, line, prod, j] - q[tank, line, prod, j] * Bounds[line]["u"] <= 0)
            model.addConstr(o[tank, line, prod, j] - q[tank, line, prod, j] * Bounds[line]["l"] >= 0) 

        #-------------------------------------------------------------------------------------------------------------                        
        # Zero outflows 
        #    
        lst = list(set([t[0:4] for t in o_index]))            
        for tup in lst:
            tank = tup[0]; line = tup[1]; prod = tup[2]; j = tup[3]
            value = CycleVolOut2.get(line, {}).get(prod)
            if value is None:
                model.addConstr(q[tank, line, prod, j] == 0)
                model.addConstr(o[tank, line, prod, j] == 0)
        
        return (model)

    
    
    

    @staticmethod
    def model_timeline(ID, model, inputs):
        
        """
        Applies constraints to the given mathematical optimization `model` based on specific time constraints from the provided `inputs`.

        This function restricts the values of production (`p`) and consumption (`q`) variables based on time constraints
        defined in an external CSV file. It uses the timeline data from the CSV file to limit the values of `p` and `q`
        for specific tanks, lines, products, and time steps.

        Parameters:
            ID (int): An identifier or key used to locate the corresponding timeline CSV file.
            model (OptimizationModel): The mathematical optimization model to which constraints are added.
            inputs (dict): A dictionary containing input data required to create constraints.
                The dictionary should have the following keys:
                    - 'index': A dictionary containing information about 'i' and 'o' indexes.
                        It should have the following keys:
                            - 'i': A list of tuples representing the 'i' index with format (tank, line, prod, time, ...)
                            - 'o': A list of tuples representing the 'o' index with format (tank, line, prod, time, ...)
                    - 'p': The optimization variable 'p' representing production in the model.
                    - 'i': The optimization variable 'i' representing the inflow of tanks in the model.
                    - 'q': The optimization variable 'q' representing consumption in the model.

        Returns:
            OptimizationModel: The `model` with added constraints based on the timeline data.

        Example:
            # Assume 'model' and 'inputs' are defined.
            ID = 123  # The identifier for the corresponding timeline CSV file.
            model = function_model_const9(ID, model, inputs)

        Note:
            - The CSV file containing timeline data should be named 'opt_timeline_<ID>.csv', where <ID> is the given `ID`.
            - The timeline data should include information about lines, products, and their time constraints.
            - The function will read the timeline data from the CSV file and apply constraints on `p` and `q` variables
            based on the time limits specified in the timeline data.
            - If the time step of a variable falls outside the specified time limits for a given line and product,
            the function sets the corresponding variable to zero in the optimization model, effectively restricting
            the production or consumption at that specific time step and location.
            - The function ensures that production and consumption occur within the valid time ranges specified in the timeline data.
        """
        
        i_index, o_index, p, i, o, q = (
            inputs['index']['i'],
            inputs['index']['o'],
            inputs['p'],
            inputs['i'],
            inputs['o'],
            inputs['q']
        )    

        opt_1_timeline = pd.read_csv ('results/opt_timeline_' + str(ID) + '.csv')
        def line(df):
            if (df['Line'] in [1]):
                return '01'
            elif (df['Line'] in [2]):
                return '02'
            else:
                return str(df['Line'])
        opt_1_timeline['Line'] = opt_1_timeline.apply(line, axis = 1)

        lst = list(set([t[0:4] for t in i_index])) 
        for tup in lst:
            tank = tup[0]; line = tup[1]; prod = tup[2]; time = tup[3]
            a = opt_1_timeline[(opt_1_timeline['Line'] == line) & (opt_1_timeline['Product'] == prod)]   
            if (len(a.index) > 0):
                if ((time < a.iloc[0]["Time_min_f"]) | (time > a.iloc[0]["Time_max_f"])):
                    model.addConstr(p[tank, line, prod, time] == 0)
                    model.addConstr(i[tank, line, prod, time] == 0)

        lst = list(set([t[0:4] for t in o_index])) 
        for tup in lst:
            tank = tup[0]; line = tup[1]; prod = tup[2]; time = tup[3]
            a = opt_1_timeline[(opt_1_timeline['Line'] == line) & (opt_1_timeline['Product'] == prod)]   
            if (len(a.index) > 0):
                if ((time < a.iloc[0]["Time_min_f"]) | (time > a.iloc[0]["Time_max_f"])):
                    model.addConstr(q[tank, line, prod, time] == 0)
                    model.addConstr(o[tank, line, prod, time] == 0)
                            
        return (model)  
    
    @staticmethod
    def model_sequencing(model, inputs):

        """
        Adds specific constraints to the given mathematical optimization `model` based on the provided `inputs`.

        This function adds constraints to the optimization model for tank-to-line operations (`mo`) and line-to-tank
        operations (`mi`). It restricts the simultaneous flow between certain tanks and lines to ensure consistency
        and meet the constraints of the system.

        Parameters:
            model (OptimizationModel): The mathematical optimization model to which constraints are added.
            inputs (dict): A dictionary containing input data required to create constraints.
                The dictionary should have the following keys:
                    - 'index': A dictionary containing information about 'i', 'o', and 'o' indexes.
                        It should have the following keys:
                            - 'i': A list of tuples representing the 'i' index with format (tank, line, prod, time, ...)
                            - 'o': A list of tuples representing the 'o' index with format (tank, line, prod, time, ...)
                    - 'mo': The optimization variable 'mo' representing tank-to-line operations in the model.
                    - 'mi': The optimization variable 'mi' representing line-to-tank operations in the model.
                    - 'p': The optimization variable 'p' representing production in the model.
                    - 'q': The optimization variable 'q' representing consumption in the model.
                    - 'T': The total number of time steps in the model.

        Returns:
            OptimizationModel: The `model` with added constraints.

        Example:
            # Assume 'model' and 'inputs' are defined.
            model = function_model_const6(model, inputs)

        Note:
            - The 'mo' and 'mi' optimization variables must be previously defined in the model before calling this function.
            - The function will create constraints for tank-to-line operations (`mo`) based on the provided 'o' index and time-dependent information.
            - The function will create constraints for line-to-tank operations (`mi`) based on the provided 'i' index and time-dependent information.
            - The constraints ensure that simultaneous flow between certain tanks and lines follows specific rules.
            - The function applies different constraints for specific lines (e.g., '01', '02', '13', '14', etc.) and products (e.g., 'A', 'D', '54').
            - The constraints aim to ensure the consistency of the system and meet the specified capacity constraints.
        """

        i_index, o_index, mo, mi, p, q, T = (
            inputs['index']['i'],
            inputs['index']['o'],
            inputs['mo'],
            inputs['mi'],
            inputs['p'],
            inputs['q'],
            inputs['T']
        )

        #-------------------------------------------------------------------------------------------------------------
        #Constraint 9: Marker
        # 
        lst = [tup[0:3] for tup in o_index if tup[3] == 0]
        for tpl in lst:
            tank = tpl[0]; line = tpl[1]; prod = tpl[2] 
            for j in list(range(T)):
                for k in list(range(j + 1)):
                    model.addConstr(mo[line, prod, j] >= q[tank, line, prod, k])

        lst = [tup[0:3] for tup in i_index if tup[3] == 0]
        for tpl in lst:
            tank = tpl[0]; line = tpl[1]; prod = tpl[2] 
            for j in list(range(T)):
                for k in list(range(j + 1)):
                    model.addConstr(mi[line, prod, j] >= p[tank, line, prod, k])   

        #-------------------------------------------------------------------------------------------------------------
        # Line - 01
        #      
        line = '01'
        #
        lst1 = [tup[0:3] for tup in i_index if tup[1] == line and tup[3] == 0 and tup[2] == 'A']
        lst2 = [tup[1:3] for tup in i_index if tup[1] == line and tup[3] == 0 and tup[2] not in ['A']]
        lst2 = list(set(lst2))
        for tpl1 in lst1:
            tank1 = tpl1[0]
            line1 = tpl1[1]
            prod1 = tpl1[2]
            for tpl2 in lst2:
                line2 = tpl2[0]
                prod2 = tpl2[1]
                for j in list(range(T)):
                    model.addConstr(p[tank1, line1, prod1, j] + mi[line2, prod2, j] <= 1) 

        #-------------------------------------------------------------------------------------------------------------
        # Line - 02
        #      
        line = '02'
        #
        lst1 = [tup[0:3] for tup in i_index if tup[1] == line and tup[3] == 0 and tup[2] == '54']
        lst2 = [tup[1:3] for tup in i_index if tup[1] == line and tup[3] == 0 and tup[2] not in ['54']]
        lst2 = list(set(lst2))
        for tpl1 in lst1:
            tank1 = tpl1[0]
            line1 = tpl1[1]
            prod1 = tpl1[2]
            for tpl2 in lst2:
                line2 = tpl2[0]
                prod2 = tpl2[1]
                for j in list(range(T)):
                    model.addConstr(p[tank1, line1, prod1, j] + mi[line2, prod2, j] <= 1)             

        #-------------------------------------------------------------------------------------------------------------
        # Line - 13
        #
        line = '13'
        prds = ['A', 'D', '54']
        
        for i in range(len(prds)):

            lst1 = [tup[0:3] for tup in o_index if tup[1] == line and tup[3] == 0 and tup[2] == prds[i]]
            lst2 = [tup[1:3] for tup in o_index if tup[1] == line and tup[3] == 0 and tup[2] not in prds[0:(i+1)]]
            lst2 = list(set(lst2))
            for tpl1 in lst1:
                tank1 = tpl1[0]
                line1 = tpl1[1]
                prod1 = tpl1[2]
                for tpl2 in lst2:
                    line2 = tpl2[0]
                    prod2 = tpl2[1]
                    for j in list(range(T)):
                        model.addConstr(q[tank1, line1, prod1, j] + mo[line2, prod2, j] <= 1)    


        #-------------------------------------------------------------------------------------------------------------
        # Line - 14
        #
        line = '14'
        prds = ['A', 'D', '54']
        
        for i in range(len(prds)):

            lst1 = [tup[0:3] for tup in o_index if tup[1] == line and tup[3] == 0 and tup[2] == prds[i]]
            lst2 = [tup[1:3] for tup in o_index if tup[1] == line and tup[3] == 0 and tup[2] not in prds[0:(i+1)]]
            lst2 = list(set(lst2))
            for tpl1 in lst1:
                tank1 = tpl1[0]
                line1 = tpl1[1]
                prod1 = tpl1[2]
                for tpl2 in lst2:
                    line2 = tpl2[0]
                    prod2 = tpl2[1]
                    for j in list(range(T)):
                        model.addConstr(q[tank1, line1, prod1, j] + mo[line2, prod2, j] <= 1)        

            

        #-------------------------------------------------------------------------------------------------------------                       
        # Line - 15
        #
        line = '15'
        prds = ['A', 'D', '54']
        
        for i in range(len(prds)):

            lst1 = [tup[0:3] for tup in o_index if tup[1] == line and tup[3] == 0 and tup[2] == prds[i]]
            lst2 = [tup[1:3] for tup in o_index if tup[1] == line and tup[3] == 0 and tup[2] not in prds[0:(i+1)]]
            lst2 = list(set(lst2))
            for tpl1 in lst1:
                tank1 = tpl1[0]
                line1 = tpl1[1]
                prod1 = tpl1[2]
                for tpl2 in lst2:
                    line2 = tpl2[0]
                    prod2 = tpl2[1]
                    for j in list(range(T)):
                        model.addConstr(q[tank1, line1, prod1, j] + mo[line2, prod2, j] <= 1) 

        #-------------------------------------------------------------------------------------------------------------                   
        # Line - 16
        #
        line = '16'
        prds = ['A', 'D', '54']
        
        for i in range(len(prds)):

            lst1 = [tup[0:3] for tup in o_index if tup[1] == line and tup[3] == 0 and tup[2] == prds[i]]
            lst2 = [tup[1:3] for tup in o_index if tup[1] == line and tup[3] == 0 and tup[2] not in prds[0:(i+1)]]
            lst2 = list(set(lst2))
            for tpl1 in lst1:
                tank1 = tpl1[0]
                line1 = tpl1[1]
                prod1 = tpl1[2]
                for tpl2 in lst2:
                    line2 = tpl2[0]
                    prod2 = tpl2[1]
                    for j in list(range(T)):
                        model.addConstr(q[tank1, line1, prod1, j] + mo[line2, prod2, j] <= 1)          

        #-------------------------------------------------------------------------------------------------------------                        
        # Line - 17
        #
        line = '17'
        prds = ['A', 'D', '54']
        
        for i in range(len(prds)):

            lst1 = [tup[0:3] for tup in o_index if tup[1] == line and tup[3] == 0 and tup[2] == prds[i]]
            lst2 = [tup[1:3] for tup in o_index if tup[1] == line and tup[3] == 0 and tup[2] not in prds[0:(i+1)]]
            lst2 = list(set(lst2))
            for tpl1 in lst1:
                tank1 = tpl1[0]
                line1 = tpl1[1]
                prod1 = tpl1[2]
                for tpl2 in lst2:
                    line2 = tpl2[0]
                    prod2 = tpl2[1]
                    for j in list(range(T)):
                        model.addConstr(q[tank1, line1, prod1, j] + mo[line2, prod2, j] <= 1)            

        #-------------------------------------------------------------------------------------------------------------                        
        # Line - 18
        #
        line = '18'
        prds = ['A', 'D', '54']
        
        for i in range(len(prds)):

            lst1 = [tup[0:3] for tup in o_index if tup[1] == line and tup[3] == 0 and tup[2] == prds[i]]
            lst2 = [tup[1:3] for tup in o_index if tup[1] == line and tup[3] == 0 and tup[2] not in prds[0:(i+1)]]
            lst2 = list(set(lst2))
            for tpl1 in lst1:
                tank1 = tpl1[0]
                line1 = tpl1[1]
                prod1 = tpl1[2]
                for tpl2 in lst2:
                    line2 = tpl2[0]
                    prod2 = tpl2[1]
                    for j in list(range(T)):
                        model.addConstr(q[tank1, line1, prod1, j] + mo[line2, prod2, j] <= 1)  

        #-------------------------------------------------------------------------------------------------------------                        
        # Line - 19
        #
        line = '19'
        prds = ['A', 'D', '54']
        
        for i in range(len(prds)):

            lst1 = [tup[0:3] for tup in o_index if tup[1] == line and tup[3] == 0 and tup[2] == prds[i]]
            lst2 = [tup[1:3] for tup in o_index if tup[1] == line and tup[3] == 0 and tup[2] not in prds[0:(i+1)]]
            lst2 = list(set(lst2))
            for tpl1 in lst1:
                tank1 = tpl1[0]
                line1 = tpl1[1]
                prod1 = tpl1[2]
                for tpl2 in lst2:
                    line2 = tpl2[0]
                    prod2 = tpl2[1]
                    for j in list(range(T)):
                        model.addConstr(q[tank1, line1, prod1, j] + mo[line2, prod2, j] <= 1)  
                    
        #-------------------------------------------------------------------------------------------------------------                        
        # Line - 20
        #
        line = '20'
        prds = ['A', 'D', '54']
        
        for i in range(len(prds)):

            lst1 = [tup[0:3] for tup in o_index if tup[1] == line and tup[3] == 0 and tup[2] == prds[i]]
            lst2 = [tup[1:3] for tup in o_index if tup[1] == line and tup[3] == 0 and tup[2] not in prds[0:(i+1)]]
            lst2 = list(set(lst2))
            for tpl1 in lst1:
                tank1 = tpl1[0]
                line1 = tpl1[1]
                prod1 = tpl1[2]
                for tpl2 in lst2:
                    line2 = tpl2[0]
                    prod2 = tpl2[1]
                    for j in list(range(T)):
                        model.addConstr(q[tank1, line1, prod1, j] + mo[line2, prod2, j] <= 1)  
            
        #-------------------------------------------------------------------------------------------------------------                        
        # Line - 1A
        #
        line = '1A'
        #
        lst1 = [tup[0:3] for tup in o_index if tup[1] == line and tup[3] == 0 and tup[2] == 'A']
        lst2 = [tup[1:3] for tup in o_index if tup[1] == line and tup[3] == 0 and tup[2] not in ['A']]
        lst2 = list(set(lst2))
        for tpl1 in lst1:
            tank1 = tpl1[0]
            line1 = tpl1[1]
            prod1 = tpl1[2]
            for tpl2 in lst2:
                line2 = tpl2[0]
                prod2 = tpl2[1]
                for j in list(range(T)):
                    model.addConstr(q[tank1, line1, prod1, j] + mo[line2, prod2, j] <= 1) 
                    
        #-------------------------------------------------------------------------------------------------------------                   
        # Line - 2A
        #
        line = '2A'
        #
        lst1 = [tup[0:3] for tup in o_index if tup[1] == line and tup[3] == 0 and tup[2] == '54']
        lst2 = [tup[1:3] for tup in o_index if tup[1] == line and tup[3] == 0 and tup[2] not in ['54']]
        lst2 = list(set(lst2))
        for tpl1 in lst1:
            tank1 = tpl1[0]
            line1 = tpl1[1]
            prod1 = tpl1[2]
            for tpl2 in lst2:
                line2 = tpl2[0]
                prod2 = tpl2[1]
                for j in list(range(T)):
                    model.addConstr(q[tank1, line1, prod1, j] + mo[line2, prod2, j] <= 1)             
                    

        return (model)